<a href="https://colab.research.google.com/github/Arun-nexus/nexux/blob/main/transfer_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.applications import VGG19


In [9]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
import numpy as np

In [11]:
x_train=x_train[:20000]
y_train=y_train[:20000]


In [12]:
pre_model=VGG19(
    include_top=False,
    input_shape=(128,128,3)
)
pre_model.trainable=False

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
import cv2

In [14]:
def process(images,size=(128,128)):
  resized=[]
  for img in images:
    img=cv2.resize(img,size)
    img=cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    resized.append(img)
  return np.array(resized)

x_train=process(x_train)
x_test=process(x_test)

In [15]:
from tensorflow.keras.utils import to_categorical

In [16]:
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout

In [18]:
model=Sequential()
model.add(pre_model)
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(10,activation="softmax"))

In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg19 (Functional)              │ (None, 4, 4, 512)      │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,124,362 (84.40 MB)

 Trainable params: 2,099,978 (8.01 MB)

 Non-trainable params: 20,024,384 (76.39 MB)

In [20]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [21]:
model.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),metrics=["accuracy"])

In [22]:
from tensorflow.keras.callbacks import EarlyStopping
stopper=EarlyStopping(
    monitor="val_accuracy",
    restore_best_weights=True,
    patience=3,
    verbose=1
)

In [23]:
x_train.shape

(20000, 128, 128, 3)

In [ ]:
hist=model.fit(x_train,y_train,validation_split=0.2,epochs=100,callbacks=stopper,batch_size=256)

Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 0.8857 - loss: 0.5202 - val_accuracy: 0.9555 - val_loss: 0.2068
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 89s 649ms/step - accuracy: 0.9302 - loss: 0.2425 - val_accuracy: 0.9630 - val_loss: 0.1664
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 43s 682ms/step - accuracy: 0.9386 - loss: 0.1852 - val_accuracy: 0.9735 - val_loss: 0.1699
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 82s 676ms/step - accuracy: 0.9544 - loss: 0.1616 - val_accuracy: 0.9735 - val_loss: 0.1796
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 43s 682ms/step - accuracy: 0.9623 - loss: 0.1303 - val_accuracy: 0.9747 - val_loss: 0.1679
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 80s 644ms/step - accuracy: 0.9615 - loss: 0.1138 - val_accuracy: 0.9760 - val_loss: 0.1649
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 41s 649ms/step - accuracy: 0.9627 - loss: 0.1112 - val_accuracy: 0.9778 - val_loss: 0.1740
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 43s 677ms/step - accuracy: 0.9645 - loss: 0.1100 - val

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history["accuracy"],label="accuracy")
plt.plot(hist.history["val_accuracy"],label="val_accuracy")
plt.legend()
plt.show()

In [ ]:
from google.colab import files
files.upload()

In [ ]:
from PIL import Image
new_image=Image.open("/content/lion-Kenya-Masai-Mara-National-Reserve.webp")

In [ ]:
new_image=np.array(new_image)
new_image=new_image.resize(128,128,3)

In [ ]:
pred=model.predict(new_image)
pred=np.argmax(pred)
